In [12]:
import gensim
import numpy as np

In [4]:
model = gensim.models.KeyedVectors.load_word2vec_format('./model/GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
type(model.wv)

/Users/kenalo37/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


gensim.models.keyedvectors.Word2VecKeyedVectors

In [40]:
word_list = ['hi', 'bye', ['abusive', 'control'], 'boy', 'anxiety', 'assault', ['rape', 'them']]
w2v = np.zeros([len(word_list), 6, 300])

c1 = 0
for word in word_list:
    x = np.zeros([6, 300])
    if type(word) == list:
        c2 = 0
        for w in word:
            x[c2] = model[w]
            c2 += 1
    elif type(word) == str:
        x[0] = model[word]
    w2v[c1] = x
    c1 += 1
        

In [41]:
print(w2v)

[[[-0.16503906  0.20410156  0.13085938 ... -0.39257812 -0.19238281
    0.05786133]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.03027344  0.06542969  0.09228516 ... -0.56640625 -0.0062561
    0.15136719]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.38476562 -0.04956055 -0.03125    ... -0.00086212 -0.12890625
    0.16894531

In [33]:
x = np.zeros([6, 300])
x[2:x.shape[0], :].shape
type(word_list[0])

str

In [34]:
for word in word_list:
    x = np.zeros([6, 300])
    if type(word) == list:
        for w in word:
            print(w)
    elif type(word) == str:
        print(word)

hi
bye
abusive
control
boy
anxiety
assault
rape
them


In [48]:
## prepare data for training

# read semi-clean data
import pandas as pd
doc_complete = pd.read_csv('/Users/kenalo37/hackfest/behavior.csv')
doc_complete = list(doc_complete.iloc[:,1])

doc = [d.split(' ') for d in doc_complete]
for i in range(len(doc)):
    temp = [x for x in doc[i] if x!= '' and x.isalpha()]
    doc[i] = temp

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    """lemmatize and clean doc"""
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

lemmatized_list = []
for d in doc:
    assert type(d) == list
    if type(d) == list:
        temp = [clean(x) for x in d]
        lemmatized_list.append(temp)

for i in range(len(lemmatized_list)):
    temp = [x for x in lemmatized_list[i] if x!= '' and x.isalpha()]
    lemmatized_list[i] = temp

In [49]:
lemmatized_list

[['chinese', 'fire', 'drill'],
 ['abusive', 'cycle'],
 ['action', 'assembly', 'theory'],
 ['active'],
 ['adoption'],
 ['age'],
 ['aggresion', 'towards', 'object'],
 ['aggressive'],
 ['alcohol', 'use'],
 ['alienation'],
 ['alloplastic', 'adaptation'],
 ['always', 'never', 'statement'],
 ['ambitious'],
 ['anger'],
 ['animal', 'abuse'],
 ['animal', 'spirit'],
 ['argumentative'],
 ['asccismus'],
 ['assault', 'gun'],
 ['assault', 'knife'],
 ['assertive'],
 ['attention', 'deficit', 'hyperactivity', 'disorder'],
 ['attention', 'seeking'],
 ['attitude', 'change'],
 ['autoplastic', 'defense', 'mechanism'],
 ['avoidance'],
 ['avoidance', 'coping'],
 ['bad', 'coping'],
 ['bad', 'habit'],
 ['baiting'],
 ['beat'],
 ['beat'],
 ['behavior'],
 ['behavior', 'change'],
 ['behavioral', 'confirmation'],
 ['behavioral', 'contagion'],
 ['behavioral', 'modernity'],
 ['behavioral', 'urbanism'],
 ['belittle', 'partner'],
 ['belittling', 'condescending', 'patronizing'],
 ['bend', 'finger'],
 ['binge', 'drinking

In [56]:
def input_volume(word_list, timesteps):
    w2v = np.zeros([len(word_list), timesteps, 300])
    c1 = 0
    for word in word_list:
        x = np.zeros([6, 300])
        if type(word) == list:
            c2 = 0
            for w in word:
                try: 
                    x[c2] = model[w]
                    c2 += 1
                except KeyError:
                    pass
        elif type(word) == str:
            x[0] = model[word]
        w2v[c1] = x
        c1 += 1
    return w2v

In [57]:
input_vol = input_volume(lemmatized_list, 6)

In [60]:
np.save('input_vol', input_vol)